In [1]:
import json
import math

In [11]:
for cat in ["ae", "lp", "sc"]:
    print(cat)
    # s1_path = f"data/scores/nopers/{cat}_test.json"
    # s1_path = f"data/scores/rag/full_profile/{cat}_rag_full_profile_test_10_limit_0_scores.json"
    # s2_path = f"data/scores/rag/full_profile/{cat}_rag_full_profile_test_10_limit_0_scores.json"

    # s1_path = "data/scores/rag/full_profile/sc_rag_full_profile_validation_10_limit_0_scores.json"
    # s2_path = "data/scores/rag/full_profile/sc_rag_full_profile_validation_10_limit_1_scores.json"

    s1_path = f"data/scores/nopers/{cat}_0_validation.json"

    s2_path = f"data/scores/rag/full_profile/{cat}_rag_full_profile_validation_10_limit_0_scores.json"
    # s2_path = f"data/scores/rag/full_profile/{cat}_rag_full_profile_test_10_limit_10_scores.json"

    with open(s1_path, "r") as f:
        s1 = json.load(f)
        s1_macro_score = s1["score"]
        s1 = s1['per_question_scores']

    with open(s2_path, "r") as f:
        s2 = json.load(f)
        s2_macro_score = s2["score"]
        s2 = s2['per_question_scores']

    print(f"Macro Score {s1_macro_score:.4f} to {s2_macro_score:.4f}, {(s2_macro_score - s1_macro_score)} change")

    # ensure ids match
    for obj1, obj2 in zip(s1, s2):
        if obj1["id"] != obj2["id"]:
            raise ValueError("IDs don't match")
            break
    print("All good")

    # measures improvements from s1 to s2
    increase = 0
    decrease = 0
    tie = 0

    tie_threshold = 0.01

    for obj1, obj2 in zip(s1, s2):
        o1 = obj1["score"]
        o2 = obj2["score"]

        if abs(o2 - o1) < tie_threshold:
            tie += 1
        elif o2 > o1:
            increase += 1
        else:
            decrease += 1

    print("Improvements from S1 to S2:")
    print(f"Score increased {increase} many times")
    print(f"Score decreased {decrease} many times")
    print(f"Score was a tie {tie} many times, with {tie_threshold} margin")
    print(f"Increase % {increase / len(s1) * 100:.4f}")
    print(f"Decrease % {decrease / len(s1) * 100:.4f}")

ae
Macro Score 0.3383 to 0.2581, -0.08020734153432074 change
All good
Improvements from S1 to S2:
Score increased 62 many times
Score decreased 152 many times
Score was a tie 127 many times, with 0.01 margin
Increase % 18.1818
Decrease % 44.5748
lp
Macro Score 0.4832 to 0.4203, -0.06286094989081759 change
All good
Improvements from S1 to S2:
Score increased 91 many times
Score decreased 193 many times
Score was a tie 168 many times, with 0.01 margin
Increase % 20.1327
Decrease % 42.6991
sc
Macro Score 0.4635 to 0.4265, -0.037070368867244075 change
All good
Improvements from S1 to S2:
Score increased 77 many times
Score decreased 112 many times
Score was a tie 131 many times, with 0.01 margin
Increase % 24.0625
Decrease % 35.0000


In [10]:
for cat in ["ae"]:
    print(cat)
    s1_path = f"data/scores/nopers/{cat}_0_validation.json"
    s1_out_path = f"data/out/nopers/{cat}_validation_0_output.json"

    s_input = f"data/processed/{cat}_processed_validation.json"

    s2_path = f"data/scores/rag/full_profile/{cat}_rag_full_profile_validation_10_limit_0_scores.json"
    s2_out_path = f"data/out/rag/full_profile/{cat}_rag_full_profile_validation_10_limit_0_output.json"

    with open(s1_path, "r") as f:
        s1 = json.load(f)
        s1_macro_score = s1["score"]
        s1 = s1['per_question_scores']
    
    with open(s1_out_path) as f:
        s1_out = json.load(f)
    
    with open(s_input) as f:
        s_input = json.load(f)

    with open(s2_path, "r") as f:
        s2 = json.load(f)
        s2_macro_score = s2["score"]
        s2 = s2['per_question_scores']
    
    with open(s2_out_path) as f:
        s2_out = json.load(f)

    for i, obj in enumerate(s1):
        s1[i]["output"] = s1_out[obj["id"]][0]["output"]
        s1[i]["prompt"] = s1_out[obj["id"]][0]["prompt"]
        s1[i]["rubric_aspects"] = s_input[i]["rubric_aspects"]

    for i, obj in enumerate(s2):
        s2[i]["output"] = s2_out[obj["id"]][0]["output"]
        s2[i]["prompt"] = s2_out[obj["id"]][0]["prompt"]
        s2[i]["rubric_aspects"] = s_input[i]["rubric_aspects"]

    print(f"Macro Score {s1_macro_score:.4f} to {s2_macro_score:.4f}, {(s2_macro_score - s1_macro_score)} change")

    # ensure ids match
    for obj1, obj2 in zip(s1, s2):
        if obj1["id"] != obj2["id"]:
            raise ValueError("IDs don't match")
            break

    increase = []
    decrease = []
    tie_threshold = 0.01

    for obj1, obj2 in zip(s1, s2):
        o1 = obj1["score"]
        o2 = obj2["score"]

        if abs(o2 - o1) < tie_threshold:
            continue
        elif o2 > o1:
            increase.append((obj1, obj2))
        else:
            decrease.append((obj1, obj2))

ae
Macro Score 0.3383 to 0.2581, -0.08020734153432074 change


In [11]:
def compare_details(couple):
    print("Review the following two responses from two different personalization methods. The first approach does no-personalization, one-shot answering. The second approach does asymmetric personalization, retrieving user's posts in different domains than the target domain.")
    print("You should reason about why the score has increased or decreased between the methods.")
    print("The following will give you the prompt, the answer of each method. Followed by rubric aspects and the score recieved by each method.")

    print(couple[0]["id"])
    print(f"First Prompt: {couple[0]['prompt']}\n\nFirst Output: {couple[0]['output']}\n\n")
    print(f"Second Prompt: {couple[1]['prompt']}\n\nSecond Output: {couple[1]['output']}\n\n")


    print("Rubric scores:")
    i = 0
    for ra1, ra2 in zip(couple[0]["details"], couple[1]["details"]):
        if ra1["score"] != ra2["score"]:
            print(f"Asepct: {ra1['aspect']}")
            print(f"Reason: {couple[0]['rubric_aspects'][i]['reason']}")
            print(f"No-personalization score: {ra1['score']}, Asymmetric score: {ra2['score']}")
    
    print("Keep your answers to the point.")

In [13]:
compare_details(decrease[20])

Review the following two responses from two different personalization methods. The first approach does no-personalization, one-shot answering. The second approach does asymmetric personalization, retrieving user's posts in different domains than the target domain.
You should reason about why the score has increased or decreased between the methods.
The following will give you the prompt, the answer of each method. Followed by rubric aspects and the score recieved by each method.
0_1_109
First Prompt: <|im_start|>system
You are a helpful assistant designed to generate personalized responses to user questions.  Your output should be a valid json object in ```json ``` block that contains the following fields:
   - personalized_answer: contains the personalized answer to the user's current question.<|im_end|>
<|im_start|>user
How to efficiently farm iron nuggets?<|im_end|>
<|im_start|>assistant


First Output: To efficiently farm iron nuggets, start by identifying iron ore-rich areas in yo